In [4]:
import json
import numpy as np
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
# load loss files
loss_files = glob('./losses/*.json')

toy_losses = []
mnist_losses = []
for lf in loss_files:
    if 'mnist' in lf:
        mnist_losses.append(lf)
    else:
        toy_losses.append(lf)

In [66]:
def compute_params(loss):
    print('mean =', loss.mean(axis=1))
    print('max =', loss.max(axis=1))
    print('min =', loss.min(axis=1))
    print('last =', loss[:, -1])
    mean_5 = loss[:, :5].mean(axis=1)
    print('mean(5) =', mean_5)
    print('# exceeds = ', (loss[:, 3:].T > mean_5).sum(axis=0))
    print()

## Inspect Convergence on Toy Data

In [67]:
for tl in toy_losses:
    with open(tl, 'r') as f:
        losses = json.load(f)
    
    if 'flow' in tl:
        for key in losses:
            print(key)
            loss = np.array(losses[key])
            compute_params(loss)
    else:
        for key in losses:
            print(key)
            loss = np.array([
                losses[key][i]['elbo']
                for i in range(len(losses[key]))
            ])
            compute_params(loss)

train
mean = [2.95554038 2.98545051 2.96365576 2.94841223 2.98545995 2.9419604
 2.95388198 3.00324924 7.65833073 2.95151564]
max = [   3.74518478    3.54164968    3.58542365    3.65404981    3.61361589
    3.52818042    3.52135135    3.56527538 2334.03438832    3.56640281]
min = [2.93332391 2.95453875 2.93184366 2.9273356  2.95254712 2.90739895
 2.93095422 2.95479911 2.94219686 2.92943789]
last = [2.93796682 2.99203493 2.93924613 2.92939291 2.96003672 2.90930931
 2.93943095 2.96996036 2.95479701 2.93180678]
mean(5) = [3.29995626 3.22684613 3.3170087  3.29159037 3.26231076 3.2247066
 3.22360249 3.23905834 3.27990787 3.2214326 ]
# exceeds =  [0 0 0 0 0 0 0 1 3 0]

val
mean = [2.95438707 2.98442274 2.96440561 2.95095574 2.98605574 2.94143532
 2.95483955 3.00431646 2.99935022 2.95229225]
max = [3.42331596 3.24962429 3.29465312 3.83606534 3.3868674  3.31564194
 3.33948984 3.27854978 4.28586253 3.28817188]
min = [2.93227437 2.9486168  2.93242997 2.92654    2.94915425 2.90457038
 2.93160363 2

## Inspect Convergence on MNIST 